In [44]:
import pandas as pd
import numpy as np
import re
import string
import random

from pythainlp.util import normalize, isthai
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### TFIDF vector

In [83]:
numberReview = 100

In [84]:
review = pd.read_csv("../data/file/review/review_500.csv")
review.head(5)

,placeName,review
0,เซ็นทรัลเวิลด์,ห้างใหญ่มาก อย่างที่กล่าวขานก็คือเดินจนหลง มาก...
1,เซ็นทรัลเวิลด์,ศูนย์การค้า Central wOrld 🛒🛍️ เป็นศูนย์การค้า ...
2,เซ็นทรัลเวิลด์,ช่วงเทศกาลเที่ยวกันอย่าประมาท หน้ากากอนามัยถอด...
3,เซ็นทรัลเวิลด์,เดินทางง่ายสะดวก ไม่ว่าจะเป็นรถส่วนตัว แท็กซี่...
4,เซ็นทรัลเวิลด์,สนุกกับช่วงเวลาแห่งการเฉลิมฉลองกับครอบครัวและค...


In [85]:
# Nan count data
sum(review['review'].isnull())

1480

In [86]:
review['review'] = review['review'].astype(str)

In [94]:
def isGoogleTranslateAndNaN(text: str):
    # Google Translae and nan value
    if (text.find("(แปลโดย Google)") > -1 or text=='nan' or text==""):
        return np.nan
    return text

def filterPlaceFromReviewNumber(review : pd.DataFrame, numberReview = 500):
    placeName = (review['placeName'].value_counts()[review['placeName'].value_counts() >= numberReview]).keys()
    
    review['placeName'] = review['placeName'].map(lambda x : x if x in placeName else np.nan)
    review.dropna(inplace=True)
    return review

def removeEmoji(string):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    return emoj.sub(r'', string)

def cleanText(text:str):
    """
    Normalize Function
    - Remove zero-width spaces
    - Remove duplicate spaces
    - Reorder tone marks and vowels to standard order/spelling
    - Remove duplicate vowels and signs
    - Remove duplicate tone marks
    - Remove dangling non-base characters at the beginning of text
    """
    text = removeEmoji(text)
    text = normalize(text)

    # remove <>, #, punctutaion in order
    text = re.sub(r'<.*?>','', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[a-z|A-Z|\d|(\a-z)]', '', text)
    text = re.sub(r'…', '', text)
    text = re.sub(r'ๆ', '', text)
    # URL pattern
    text = re.sub(r"^[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$", '', text)
    for c in string.punctuation:
        text = re.sub(r'\{}'.format(c), '', text)

    return text

def text_tokenizer(text:str):
    return [term for term in word_tokenize(text, engine='attacut') if isthai(term)]

def preprocessText(review: pd.DataFrame):
    review['review'] = review['review'].map(lambda x : isGoogleTranslateAndNaN(str(x)))
    review.dropna(inplace=True)
    review = filterPlaceFromReviewNumber(review, numberReview=numberReview)

    review['review'] = review['review'].map(lambda x : cleanText(x))

    return review

In [95]:
# Manage data
reviewClean = preprocessText(review=review)
print(f"Place left {len(reviewClean['placeName'].value_counts())} unit")
reviewClean

Place left 64 unit


,placeName,review
0,เซ็นทรัลเวิลด์,ห้างใหญ่มากอย่างที่กล่าวขานก็คือเดินจนหลงมากี่...
1,เซ็นทรัลเวิลด์,ศูนย์การค้า️เป็นศูนย์การค้าที่มีชื่อเดิมเรียกว...
2,เซ็นทรัลเวิลด์,ช่วงเทศกาลเที่ยวกันอย่าประมาทหน้ากากอนามัยถอดย...
3,เซ็นทรัลเวิลด์,เดินทางง่ายสะดวกไม่ว่าจะเป็นรถส่วนตัวแท็กซี่รถ...
4,เซ็นทรัลเวิลด์,สนุกกับช่วงเวลาแห่งการเฉลิมฉลองกับครอบครัวและค...
...,...,...
32597,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,ศิลปะร่มสมัย
32598,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,เดินเพลิน
32599,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,สุดยอดครับ
32600,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,สะอาดสวยงาม


In [182]:
reviewClean[reviewClean['placeName'] == "พระบรมมหาราชวัง"]

,placeName,review
1000,พระบรมมหาราชวัง,มารถยนต์สะดวกดีโดยนำรถไปตอดที่สนามหลวงมีที่รับ...
1001,พระบรมมหาราชวัง,ไม่เคยมีครั้งใดที่ไปเที่ยวชมวัดพระแก้วและพระบร...
1002,พระบรมมหาราชวัง,วัดพระแก้ววัดพระศรีรัตนศาสดารามเป็นสถานที่ศักด...
1003,พระบรมมหาราชวัง,ไหว้พระวัดพระแก้วคือในสวยมากกกกกดีใจมากที่สักค...
1004,พระบรมมหาราชวัง,การมาสักการะพระแก้วมรกตซึ่งเป็นพระพุทธรูปที่สำ...
...,...,...
1495,พระบรมมหาราชวัง,เข้าไปกราบพระแก้วมรกตมีดอกบัวธูปเทียนบริการถอด...
1496,พระบรมมหาราชวัง,วันนีมาไหว้พระแก้วและศาลหลักเมืองเพื่อเป็นศิริ...
1497,พระบรมมหาราชวัง,วัดพระแก้วสวยงามมากถึงแม้ว่าอากาศด้านนอกจะร้อน...
1498,พระบรมมหาราชวัง,เข้าจุดคัดกรองทางด้านศาลหลักเมืองจะเดินไม่ไกลก...


In [97]:
reviewClean["placeName"].value_counts()

เซ็นทรัลเวิลด์                                      500
ริเวอร์ซิตี้ แบงค็อก                                500
สวนวชิรเบญจทัศ (สวนรถไฟ)                            500
ตลาดนกฮูก                                           500
สถานตากอากาศบางปู                                   500
                                                   ... 
แวร์เฮาส์ 30                                        179
มาดามทุสโซ                                          167
พิพิธภัณฑสถานแห่งชาติ หอศิลป์                       159
พิพิธภัณฑ์ชาวบางกอก                                 152
หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ    102
Name: placeName, Length: 64, dtype: int64

In [98]:
stop_words = [t.encode('utf-8') for t in list(thai_stopwords())]

tfidf_vector = TfidfVectorizer(
    tokenizer=text_tokenizer,
    ngram_range=(2,3),
    stop_words=stop_words,
    max_features=5000
)

In [99]:
X = tfidf_vector.fit_transform(reviewClean['review'])

c:\Users\Pongs\miniconda3\envs\TripRecommendation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [110]:
tfidf_vector.get_feature_names_out()

array(['กระเป๋า รองเท้า', 'กราบ ขอ', 'กราบ ขอ พร', ..., 'ไหว้ พระ ขอ',
       'ไหว้ ศาล', 'ไหว้ ศาล หลักเมือง'], dtype=object)

In [111]:
term_tfidf = pd.DataFrame(
    X.toarray(),
    columns=tfidf_vector.get_feature_names(),
    index=reviewClean.index
)

c:\Users\Pongs\miniconda3\envs\TripRecommendation\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [112]:
term_tfidf

,กระเป๋า รองเท้า,กราบ ขอ,กราบ ขอ พร,กราบ พระ,กราบ สักการะ,กราบไหว้ ขอ,กราบไหว้ ขอ พร,กรุง ที่,กรุง รัตนโกสินทร์,กรุงเทพ ที่,...,ไว้ บริการ,ไว้ ใน,ไว้ ให้,ไหน ก็,ไหว้ ขอ,ไหว้ ขอ พร,ไหว้ พระ,ไหว้ พระ ขอ,ไหว้ ศาล,ไหว้ ศาล หลักเมือง
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
term_tfidf.sum().sort_values(ascending=False).head(20)

จอด รถ           472.674563
ที่ จอด          407.489362
ที่ จอด รถ       360.100845
ดี มาก           354.514714
เยอะ มาก         337.187414
บรรยากาศ ดี      269.788406
ของ กิน          262.318243
ที่ นี่          244.566984
ร้าน อาหาร       242.650551
ถ่าย รูป         218.500375
ออก กำลัง        217.571780
มาก ค่ะ          212.315078
สวย มาก          211.112160
มาก ครับ         210.148497
ร้าน ค้า         208.865586
มี ร้าน          205.964330
ให้ เลือก        203.670750
กำลัง กาย        201.463740
ออก กำลัง กาย    200.686109
สวยงาม มาก       192.098238
dtype: float64

In [166]:
numberReview_dict = {}
for idx, name in enumerate(list(reviewClean['placeName'].value_counts().keys())):
    numberReview_dict[name] = reviewClean['placeName'].value_counts()[idx]

In [131]:
len(idx_type_vector['art']), len(idx_type_vector['history']), len(idx_type_vector['nature']), len(idx_type_vector['shopping'])

(11, 20, 20, 20)

In [174]:
# Combination every review to 1 vector each place
# FIX THIS!

numberPlace = len(reviewClean['placeName'].value_counts())
idx = 0
tfidfList = []
start = 0

for idx, name in enumerate(reviewClean['placeName'].unique()):
    end = start + numberReview_dict[name]
    # print(start, end, numberReview_dict[name])
    i_tfidf = term_tfidf.iloc[start:end].sum() / numberReview_dict[name]

    tfidfList.append(i_tfidf.to_numpy())

    start += numberReview_dict[name]

In [175]:
tfidf_vector_dict = {}

for idx, place in enumerate(reviewClean['placeName'].unique()):
     tfidf_vector_dict[place] = tfidfList[idx]

In [254]:
detail = pd.read_csv('../data/file/detailPlace.csv')
detail['tfidf'] = np.NaN
detail['tfidf'] = detail['tfidf'].astype(object)
len(detail)

75

In [263]:
for idx in range(len(tfidf_vector_dict)):
    try:
        detail_place = detail.iloc[idx, 1]
        # print(detail_place)
        value = tfidf_vector_dict[detail_place]
        detail.iloc[idx, -1] = value.reshape(-1,1)
    except Exception as error:
        print(error)
        continue

'ไชน่าทาวน์'
'YELO House'


In [271]:
detail.dropna(subset=['tfidf'],inplace=True)
detail = detail.reset_index(drop=True)
detail['no'] = [i+1 for i in range(len(detail))]

In [283]:
detail.insert(2, 'type2', webDetail['type'])
detail.insert(2, 'placeID', webDetail['placeId'])

In [285]:
detail.to_csv('detailTFIDF.csv', index=False)

In [286]:
webDetail = pd.read_csv('../data/file/finalPlace.csv')
webDetail['use'] = np.NaN
placeName = list(detail['name'].unique())

for idx, place in enumerate(webDetail['placeName']):
    for p in placeName:
        if place==p:
            webDetail.loc[idx, "use"] = 1

webDetail.dropna(inplace=True)
webDetail = webDetail.reset_index(drop=True)

In [287]:
# Vector ['art', 'history', 'nature', 'shopping', 'special']
type_list = ['art', 'history', 'nature', 'shopping', 'special']
type_vector = np.zeros((len(webDetail), len(type_list)))

for idx in range(len(webDetail)):
    type_vector_i = np.zeros(5)
    for t in webDetail['type'][idx].split(','):
        for i, t_i in enumerate(type_list):
            if t==t_i:
                type_vector_i[i]=1
    
    type_vector[idx] = type_vector_i

idx_type_vector = {
    "art":[],
    "history":[],
    "nature":[],
    "shopping":[],
    "special":[]
}

for idx, type_vector_i in enumerate(type_vector):
    for i, x in enumerate(type_vector_i):
        if x==1:
            if i==0:
                idx_type_vector["art"].append(idx)
            if i==1:
                idx_type_vector["history"].append(idx)
            if i==2:
                idx_type_vector["nature"].append(idx)
            if i==3:
                idx_type_vector["shopping"].append(idx)
            if i==4:
                idx_type_vector["special"].append(idx)

len(idx_type_vector['art']), len(idx_type_vector['history']), len(idx_type_vector['nature']), len(idx_type_vector['shopping'])

(9, 20, 20, 20)

### TFIDF score & Recommend from term score

In [288]:
detail = pd.read_csv("detailTFIDF.csv")
detail.head(3)

,no,name,placeID,type2,detail,business_status,address,geometry,phone_number,url,type,period,tfidf
0,1,เซ็นทรัลเวิลด์,ChIJ4VX0ws-e4jARBGaQ2IACrcQ,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,"4, 5 ถนน ราชดำริ แขวงปทุมวัน เขตปทุมวัน กรุงเท...","(13.7460002, 100.5399162)",+66 2 640 7000,https://maps.google.com/?cid=14171986354817230340,"shopping_mall,point_of_interest,establishment","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.000664085114051197..."
1,2,สยามพารากอน,ChIJIeWu482e4jARYymvLJqTQ58,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,991 ถ. พระรามที่ ๑ แขวงปทุมวัน เขตปทุมวัน กรุง...,"(13.7462411, 100.5347402)",+66 2 690 1000,https://maps.google.com/?cid=11476178565854079331,"shopping_mall,department_store,store,point_of_...","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.00028894443008002245, 0.0, 0.0, 0.0, 0.0, 0..."
2,3,พระบรมมหาราชวัง,ChIJPzZsMU6Z4jARQUzvk913bCo,"history,nature","{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,ถนน หน้าพระลาน แขวงพระบรมมหาราชวัง เขตพระนคร ก...,"(13.7498558, 100.4915765)",NaN,https://maps.google.com/?cid=3056950040631135297,"tourist_attraction,park,point_of_interest,esta...","{'monday': '8:30–15:30', 'tuesday': '8:30–15:3...","[0.0, 0.0003690889367503711, 0.000384252038422..."


In [289]:
print(detail['tfidf'].to_numpy().shape)
print(np.array([np.fromstring(detail['tfidf'].to_numpy()[0][1:-1], sep=',')]).shape)

(62,)
(1, 5000)


In [290]:
# matrix of tfidf score
def get_termVector(detail: pd.DataFrame):
    term_vector = np.array([np.fromstring(detail['tfidf'].to_numpy()[0][1:-1], sep=',')])

    for i in range(len(detail) - 1):
        term_vector_i = np.array([np.fromstring(detail['tfidf'].to_numpy()[i+1][1:-1], sep=',')])
        term_vector = np.append(term_vector, term_vector_i, axis=0)
    
    return term_vector

In [291]:
term_vector = get_termVector(detail)
term_vector.shape

(62, 5000)

In [292]:
term_score = cosine_similarity(term_vector)
term_score.shape

(62, 62)

In [293]:
def recommend(place):
    index = detail[detail['name'] == place]['no'].values[0] - 1
    distances = sorted(list(enumerate(term_score[index].reshape(-1))), reverse=True, key=lambda x:x[1])
    for i in distances[1:11]:
        print(detail.iloc[i[0]]['name'])

In [294]:
recommend('วัดอรุณราชวรารามราชวรมหาวิหาร (วัดแจ้ง)')

วัดพระศรีรัตนศาสดาราม (วัดพระแก้ว)
วัดสุทัศนเทพวรารามราชวรมหาวิหาร
พระบรมมหาราชวัง
วัดนาคปรก
วัดไตรมิตรวิทยารามวรวิหาร
วัดประยุรวงศาวาสวรวิหาร
วัดสระเกศราชวรมหาวิหาร (ภูเขาทอง)
วัดธรรมมงคลเถาบุญนนทวิหาร(หลวงพ่อวิริยังค์)
สนามหลวง
เอเชียทีค เดอะ ริเวอร์ฟรอนท์


## Full Recommendation system
- [x] TermScore
- [] RatingScore
- [] ReviewScore
- [] TypeScore

In [295]:
detail = pd.read_csv("detailTFIDF.csv")
detail.head(3)

,no,name,placeID,type2,detail,business_status,address,geometry,phone_number,url,type,period,tfidf
0,1,เซ็นทรัลเวิลด์,ChIJ4VX0ws-e4jARBGaQ2IACrcQ,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,"4, 5 ถนน ราชดำริ แขวงปทุมวัน เขตปทุมวัน กรุงเท...","(13.7460002, 100.5399162)",+66 2 640 7000,https://maps.google.com/?cid=14171986354817230340,"shopping_mall,point_of_interest,establishment","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.000664085114051197..."
1,2,สยามพารากอน,ChIJIeWu482e4jARYymvLJqTQ58,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,991 ถ. พระรามที่ ๑ แขวงปทุมวัน เขตปทุมวัน กรุง...,"(13.7462411, 100.5347402)",+66 2 690 1000,https://maps.google.com/?cid=11476178565854079331,"shopping_mall,department_store,store,point_of_...","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.00028894443008002245, 0.0, 0.0, 0.0, 0.0, 0..."
2,3,พระบรมมหาราชวัง,ChIJPzZsMU6Z4jARQUzvk913bCo,"history,nature","{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,ถนน หน้าพระลาน แขวงพระบรมมหาราชวัง เขตพระนคร ก...,"(13.7498558, 100.4915765)",NaN,https://maps.google.com/?cid=3056950040631135297,"tourist_attraction,park,point_of_interest,esta...","{'monday': '8:30–15:30', 'tuesday': '8:30–15:3...","[0.0, 0.0003690889367503711, 0.000384252038422..."


In [297]:
webDetail = pd.read_csv('../data/file/finalPlace.csv')
webDetail['use'] = np.NaN
placeName = list(detail['name'].unique())

for idx, place in enumerate(webDetail['placeName']):
    for p in placeName:
        if place==p:
            webDetail.loc[idx, "use"] = 1

webDetail.dropna(inplace=True)
webDetail = webDetail.reset_index(drop=True)

In [298]:
len(webDetail)

62

In [299]:
# Type from Place API (Too much type)

type_set = set()

for idx, type_i in enumerate(detail['type2']):
    for t in type_i.split(','):
        type_set.add(t)

type_list = sorted(list(type_set))
type_list

['art', 'history', 'nature', 'shopping', 'special']

In [300]:
# Vector ['art', 'history', 'nature', 'shopping', 'special']
type_vector = np.zeros((len(detail), len(type_list)))

for idx in range(len(detail)):
    type_vector_i = np.zeros(5)
    for t in detail['type2'][idx].split(','):
        for i, t_i in enumerate(type_list):
            if t==t_i:
                type_vector_i[i]=1
    
    type_vector[idx] = type_vector_i

In [301]:
detail['type2'][3]

'history,special'

In [302]:
type_vector[3]

array([0., 1., 0., 0., 1.])

In [303]:
type_score = cosine_similarity(type_vector)

type_score

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [309]:
def get_random_index(trip_level):
    # Find index of each type from type vector (Adding more History before TFIDF)
    idx_type_vector = {
        "art":[],
        "history":[],
        "nature":[],
        "shopping":[],
        "special":[]
    }

    for idx, type_vector_i in enumerate(type_vector):
        for i, x in enumerate(type_vector_i):
            if x==1:
                if i==0:
                    idx_type_vector["art"].append(idx)
                if i==1:
                    idx_type_vector["history"].append(idx)
                if i==2:
                    idx_type_vector["nature"].append(idx)
                if i==3:
                    idx_type_vector["shopping"].append(idx)
                if i==4:
                    idx_type_vector["special"].append(idx)

    print(len(idx_type_vector['art']), len(idx_type_vector['history']), len(idx_type_vector['nature']), len(idx_type_vector['shopping']))

    # Random choices place index follow Trip level
    all_random_index = set()
    # Number of place art is 4
    try:
        random_index_art = random.sample(idx_type_vector['art'], k=trip_level[0])
    except:
        random_index_art = random.sample(idx_type_vector['art'], k=trip_level[0]-1)
    random_index_history = random.sample(idx_type_vector['history'], k=trip_level[1])
    random_index_nature = random.sample(idx_type_vector['nature'], k=trip_level[2])
    random_index_shopping = random.sample(idx_type_vector['shopping'], k=trip_level[3])

    for i in random_index_art: all_random_index.add(i)
    for i in random_index_history: all_random_index.add(i)
    for i in random_index_nature: all_random_index.add(i)
    for i in random_index_shopping: all_random_index.add(i)

    all_random_index = list(all_random_index)
    
    return all_random_index


def get_term_score(all_random_index):
    # Term vector
    # Select vector from upper index
    trip_term_vector = np.zeros(term_vector[0].shape)

    for idx_vector in all_random_index:
        trip_term_vector += term_vector[idx_vector]

    trip_term_vector = trip_term_vector / len(all_random_index)
    trip_term_score = cosine_similarity(np.append(term_vector, trip_term_vector.reshape(1,-1), axis=0))
    return trip_term_score

def get_type_score(all_random_index):
    # Type vector
    trip_type_vector = np.zeros(type_vector[0].shape)

    for idx_vector in all_random_index:
        trip_type_vector += type_vector[idx_vector]

    trip_type_vector = trip_type_vector / len(all_random_index)
    trip_type_score = cosine_similarity(np.append(type_vector, trip_type_vector.reshape(1,-1), axis=0))
    return trip_type_score

In [319]:
# Trip vector level 1-5 ['art', 'history', 'nature', 'special', 'shopping']
# This input emphasize the history place
# select level unit (mulitpy 1)
trip_level = [5,1,1,1] # Level ['art', 'history', 'nature', 'shopping']

random_index = get_random_index(trip_level)
trip_term_score = get_term_score(random_index)
trip_type_score = get_type_score(random_index)

# Rating score
rating_score = webDetail['rating'].to_numpy()
rating_score = np.append(rating_score, 5)

relevance_score = trip_term_score[-1]*0.5 + trip_type_score[-1]*0.3 + rating_score*0.2

# Recommendation
distances = sorted(list(enumerate(relevance_score.reshape(-1))), reverse=True, key=lambda x:x[1])
for i in distances[1:11]:
    # print(i[1])
    print(detail.iloc[i[0]]['name'])

9 20 20 20
มิวเซียมสยาม
หอศิลปวัฒนธรรมแห่งกรุงเทพมหานคร
พิพิธภัณฑ์ศิลปะไทยร่วมสมัย
พิพิธภัณฑสถานแห่งชาติพระนคร
หอศิลป์ร่วมสมัยราชดำเนิน
นิทรรศน์รัตนโกสินทร์
บ้านศิลปินคลองบางหลวง : Artist House Bangkok
พิพิธภัณฑ์ชาวบางกอก
แวร์เฮาส์ 30
สวนหลวง ร.9
